In [1]:
import gzip
import bz2
import lzma
# from PIL.PngImagePlugin import getchunks
# from PIL import Image
import sys
from tqdm import tqdm
import torch.nn.functional as F


import io


class DefaultCompressor:
    """for non-neural-based compressor"""
    def __init__(self, compressor, typ='text'):
        if compressor == 'gzip':
            self.compressor = gzip
        elif compressor == 'bz2':
            self.compressor = bz2
        elif compressor == 'lzma':
            self.compressor = lzma
        else:
            raise RuntimeError("Unsupported compressor")
        self.type = typ
    def get_compressed_len(self, x):
        if self.type == 'text':
            return len(self.compressor.compress(x.encode('utf-8')))
        else:
            return len(self.compressor.compress(np.array(x).tobytes()))
    def get_bits_per_char(self, original_fn):
        with open(original_fn) as fo:
            data = fo.read()
            compressed_str = self.compressor.compress(data.encode('utf-8'))
            return len(compressed_str)*8/len(data)

In [2]:
import os
import torch
import numpy as np
import statistics
import operator
from collections import Counter, defaultdict
from tqdm import tqdm
import random
from functools import partial
from itertools import repeat
from copy import deepcopy
from statistics import mode
import pickle
from sklearn.metrics.cluster import adjusted_rand_score, normalized_mutual_info_score


class KnnExpText:
    def __init__(self, agg_f, comp, dis):
        self.aggregation_func = agg_f
        self.compressor = comp
        self.distance_func = dis
        self.dis_matrix = []

    def calc_dis(self, data, train_data=None, fast=False):
        if train_data is not None:
            data_to_compare = train_data
        else:
            data_to_compare = data
        for i, t1 in tqdm(enumerate(data)):
            distance4i = []
            if fast:
                t1_compressed = self.compressor.get_compressed_len_fast(t1)
            else:
                t1_compressed = self.compressor.get_compressed_len(t1)
            for j, t2 in enumerate(data_to_compare):
                if fast:
                    t2_compressed = self.compressor.get_compressed_len_fast(t2)
                    t1t2_compressed = self.compressor.get_compressed_len_fast(self.aggregation_func(t1,t2))
                else:
                    t2_compressed = self.compressor.get_compressed_len(t2)
                    t1t2_compressed = self.compressor.get_compressed_len(self.aggregation_func(t1, t2))
                distance = self.distance_func(t1_compressed, t2_compressed, t1t2_compressed)
                distance4i.append(distance)
            self.dis_matrix.append(distance4i)

    def calc_dis_with_single_compressed_given(self, data, data_len=None, train_data=None):
        if train_data is not None:
            data_to_compare = train_data
        else:
            data_to_compare = data
        for i, t1 in tqdm(enumerate(data)):
            distance4i = []
            t1_compressed = self.compressor.get_compressed_len_given_prob(t1, data_len[i])
            for j, t2 in tqdm(enumerate(data_to_compare)):
                t2_compressed = self.compressor.get_compressed_len_given_prob(t2, data_len[j])
                t1t2_compressed = self.compressor.get_compressed_len(self.aggregation_func(t1, t2))
                distance = self.distance_func(t1_compressed, t2_compressed, t1t2_compressed)
                distance4i.append(distance)
            self.dis_matrix.append(distance4i)

    def calc_dis_single(self, t1, t2):
        t1_compressed = self.compressor.get_compressed_len(t1)
        t2_compressed = self.compressor.get_compressed_len(t2)
        t1t2_compressed = self.compressor.get_compressed_len(self.aggregation_func(t1, t2))
        distance = self.distance_func(t1_compressed, t2_compressed, t1t2_compressed)
        return distance
    def calc_dis_single_multi(self, train_data, datum):
        distance4i = []
        t1_compressed = self.compressor.get_compressed_len(datum)
        for j, t2 in tqdm(enumerate(train_data)):
            t2_compressed = self.compressor.get_compressed_len(t2)
            t1t2_compressed = self.compressor.get_compressed_len(self.aggregation_func(datum, t2))
            distance = self.distance_func(t1_compressed, t2_compressed, t1t2_compressed)
            distance4i.append(distance)
        return distance4i
    def calc_dis_with_vector(self, data, train_data=None):
        if train_data is not None:
            data_to_compare = train_data
        else:
            data_to_compare = data
        for i, t1 in tqdm(enumerate(data)):
            distance4i = []
            for j, t2 in enumerate(data_to_compare):
                distance = self.distance_func(t1, t2)
                distance4i.append(distance)
            self.dis_matrix.append(distance4i)
    def calc_acc(self, k, label, train_label=None, provided_distance_matrix=None, rand=False):
        if provided_distance_matrix is not None:
            self.dis_matrix = provided_distance_matrix
        correct = []
        pred = []
        if train_label is not None:
            compare_label = train_label
            start = 0
            end = k
        else:
            compare_label = label
            start = 1
            end = k+1
        for i in range(len(self.dis_matrix)):
            sorted_idx = np.argsort(np.array(self.dis_matrix[i]))
            pred_labels = defaultdict(int)
            for j in range(start, end):
                pred_l = compare_label[sorted_idx[j]]
                pred_labels[pred_l] += 1
            sorted_pred_lab = sorted(pred_labels.items(), key=operator.itemgetter(1), reverse=True)
            most_count = sorted_pred_lab[0][1]
            if_right = 0
            most_label = sorted_pred_lab[0][0]
            most_voted_labels = []
            for pair in sorted_pred_lab:
                if pair[1] < most_count:
                    break
                if not rand:
                    if pair[0] == label[i]:
                        if_right = 1
                        most_label = pair[0]
                else:
                    most_voted_labels.append(pair[0])
            if rand:
                most_label = random.choice(most_voted_labels)
                if_right = 1 if most_label==label[i] else 0
            pred.append(most_label)
            correct.append(if_right)
        print("Accuracy is {}".format(sum(correct)/len(correct)))
        return pred, correct
    def combine_dis_acc(self, k, data, label, train_data=None, train_label=None):
        correct = []
        pred = []
        if train_label is not None:
            compare_label = train_label
            start = 0
            end = k
        else:
            compare_label = label
            start = 1
            end = k+1
        if train_data is not None:
            data_to_compare = train_data
        else:
            data_to_compare = data
        for i, t1 in tqdm(enumerate(data)):
            distance4i = self.calc_dis_single_multi(data_to_compare, t1)
            sorted_idx = np.argsort(np.array(distance4i))
            pred_labels = defaultdict(int)
            for j in range(start, end):
                pred_l = compare_label[sorted_idx[j]]
                pred_labels[pred_l] += 1
            sorted_pred_lab = sorted(pred_labels.items(), key=operator.itemgetter(1), reverse=True)
            most_count = sorted_pred_lab[0][1]
            if_right = 0
            most_label = sorted_pred_lab[0][0]
            for pair in sorted_pred_lab:
                if pair[1] < most_count:
                    break
                if pair[0] == label[i]:
                    if_right = 1
                    most_label = pair[0]
            pred.append(most_label)
            correct.append(if_right)
        print("Accuracy is {}".format(sum(correct) / len(correct)))
        return pred, correct

    def combine_dis_acc_single(self, k, train_data, train_label, datum, label):
        # Support multi processing - must provide train data and train label
        distance4i = self.calc_dis_single_multi(train_data, datum)
        sorted_idx = np.argpartition(np.array(distance4i), range(k))
        pred_labels = defaultdict(int)
        for j in range(k):
            pred_l = train_label[sorted_idx[j]]
            pred_labels[pred_l] += 1
        sorted_pred_lab = sorted(pred_labels.items(), key=operator.itemgetter(1), reverse=True)
        most_count = sorted_pred_lab[0][1]
        if_right = 0
        most_label = sorted_pred_lab[0][0]
        prob = sorted_pred_lab[0][0]
        for pair in sorted_pred_lab:
            if pair[1] < most_count:
                break
            if pair[0] == label:
                if_right = 1
                most_label = pair[0]
        pred=most_label
        correct=if_right
        return pred, correct

In [3]:
from scipy.spatial.distance import cosine
import numpy as np
import torch
import scipy

class ToInt:
    def __call__(self, pic):
        return pic * 255

def NCD(c1, c2, c12):
    dis = (c12-min(c1,c2))/max(c1, c2)
    return dis

def CLM(c1, c2, c12):
    dis = 1 - (c1+c2-c12)/c12
    return dis

def CDM(c1, c2, c12):
    dis = c12/(c1+c2)
    return dis

def MSE(v1, v2):
    return np.sum((v1-v2)**2)/len(v1)

def agg_by_concat_space(t1, t2):
    return t1+' '+t2

def agg_by_jag_word(t1, t2, trunc=True):
    t1_list = t1.split(' ')
    t2_list = t2.split(' ')
    comb = []
    l = min([len(t1_list), len(t2_list)])
    for i in range(0,l-1,2):
        comb.append(t1_list[i])
        comb.append(t2_list[i+1])
    if len(t1_list) > len(t2_list):
        comb += t1_list[i:]
    return ' '.join(comb)

def agg_by_jag_char(t1, t2, trunc=True):
    t1_list = list(t1)
    t2_list = list(t2)
    comb = []
    l = min([len(t1_list), len(t2_list)])
    for i in range(0,l-1,2):
        comb.append(t1_list[i])
        comb.append(t2_list[i+1])
    if len(t1_list) > len(t2_list):
        comb += t1_list[i:]
    return ''.join(comb)

def agg_by_avg(i1, i2):
    return torch.div(i1+i2, 2, rounding_mode='trunc')

def agg_by_min_or_max(i1, i2, func_n):
    stacked = torch.stack([i1, i2], axis=0)
    if func_n == 'min':
        return torch.min(stacked, axis=0)[0]
    else:
        return torch.max(stacked, axis=0)[0]

def agg_by_stack(i1, i2):
    return torch.stack([i1, i2])

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

In [4]:
from functools import partial
from pathos.multiprocessing import ProcessingPool as Pool
import time
# np.random.seed(6)

def non_neural_knn_exp(compressor_name, test_data, test_label, train_data, train_label, agg_func, dis_func, k, para=True):
    print("KNN with compressor={}".format(compressor_name))
    cp = DefaultCompressor(compressor_name)
    knn_exp_ins = KnnExpText(agg_func, cp, dis_func)
    start = time.time()
    if para:
        with Pool(5) as p:
            pred_correct_pair = p.map(partial(knn_exp_ins.combine_dis_acc_single, k, train_data, train_label), test_data, test_label)
        print('accuracy:{}'.format(np.average(np.array(pred_correct_pair, dtype=np.int32)[:,1])))
        # print('accuracy:{}'.format(np.average(np.array(pred_correct_pair, dtype=np.object_)[:, 1])))
    else:
        knn_exp_ins.calc_dis(test_data, train_data=train_data)
        knn_exp_ins.calc_acc(k, test_label, train_label=train_label)
    print("spent: {}".format(time.time() - start))

def record_distance(compressor_name, test_data, test_portion_name, train_data, agg_func, dis_func, out_dir, para=True):
    print("compressor={}".format(compressor_name))
#     numpy_dir = os.path.join(out_dir, compressor_name)
#     if not os.path.exists(numpy_dir):
#         os.makedirs(numpy_dir)
#     out_fn = os.path.join(numpy_dir, test_portion_name)
    cp = DefaultCompressor(compressor_name)
    knn_exp = KnnExpText(agg_func, cp, dis_func)
    start = time.time()
    if para:
        with Pool(6) as p:
            distance_for_selected_test = p.map(partial(knn_exp.calc_dis_single_multi, train_data), test_data)
#         np.save(out_fn, np.array(distance_for_selected_test))
        del distance_for_selected_test
    else:
        knn_exp.calc_dis(test_data, train_data=train_data)
#         np.save(out_fn, np.array(knn_exp.dis_matrix))
    print("spent: {}".format(time.time() - start))
    return np.array(knn_exp.dis_matrix)


def non_neurl_knn_exp_given_dis(dis_matrix, k, test_label, train_label):
    knn_exp = KnnExpText(None, None, None)
    pred, correct = knn_exp.calc_acc(k, test_label, train_label=train_label, provided_distance_matrix=dis_matrix)
    return pred, correct

In [5]:
# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    df['text'] = second_sentence
    df['answer'] = option_to_index[example['answer']]
    return df

In [6]:
import pandas as pd
train = pd.read_csv("data/ScienceQA/train.csv")
df = pd.DataFrame()
first = []
secend = []
answer = []
for index, data in train.iterrows():
    for j in options:
        first.append(data['prompt'])
        secend.append(data[j])
        if j == data['answer']:
            answer.append(1)
        else:
            answer.append(0)
            
df['first'] = first
df['secend'] = secend
df['answer'] = answer

In [7]:
test = pd.read_csv("data/ScienceQA/test.csv")

df2 = pd.DataFrame()
first = []
secend = []
answer = []
id_ = []
for index, data in test.iterrows():
    for j in options:
        id_.append(index)
        first.append(data['prompt'])
        secend.append(data[j])
        answer.append(0)
df2['id'] = id_
df2['first'] = first
df2['secend'] = secend
df2['answer'] = answer

In [8]:

# predicts = np.zeros((test.shape[0], 5))
# for i, data in test.iterrows():
#     dis_matrix = record_distance('gzip', data['prompt'], 'test_dis', np.array(data[['A', 'B', 'C', 'D', 'E']]), agg_by_jag_char, NCD, '/kaggle/working/', para=False)
#     predicts[i, :] = dis_matrix[0, :]

predicts = np.zeros((train.shape[0], 5))
for i, data in train.iterrows():
    dis_matrix = record_distance('gzip', data['prompt'], 'train_dis', np.array(data[['A', 'B', 'C', 'D', 'E']]), agg_by_jag_char, NCD, '/kaggle/working/', para=False)
    predicts[i, :] = dis_matrix[0, :]

compressor=gzip


175it [00:00, 7761.97it/s]


spent: 0.024910926818847656
compressor=gzip


92it [00:00, 7448.77it/s]


spent: 0.013438940048217773
compressor=gzip


...

spent: 0.016880273818969727
compressor=gzip


111it [00:00, 7216.09it/s]


spent: 0.01640915870666504
compressor=gzip


88it [00:00, 7734.51it/s]

spent: 0.01241755485534668


In [9]:
def predictions_to_map_output(predictions):
    sorted_answer_indices = np.argsort(predictions)
    top_answer_indices = sorted_answer_indices[:,:3] # Get the first three answers in each row
    top_answers = np.vectorize(index_to_option.get)(top_answer_indices)
    return np.apply_along_axis(lambda row: ' '.join(row), 1, top_answers)

In [10]:
train['prediction'] = predictions_to_map_output(predicts)

In [11]:
train['prediction'].tolist()

['A C E',
 'E B C',
 'A D E',
 'E A B',
 'D B A',
 'E D B',
 'E A C',
 'C A D',
 'C D E',
 'D B E',
 'A D C',
 'C E B',
 'B D E',
 'C A B',
 'E D C',
 'B C A',
 'D E B',
 'C D B',
 'C B E',
 'C A B',
 'A E C',
 'A C B',
 'D E B',
 'A D E',
 'B C D',
 'C D A',
 'D B C',
 'A B D',
 'D C A',
 'D A E',
 'A C D',
 'E B C',
 'C A D',
 'B C E',
 'A B D',
 'C A E',
 'A E B',
 'B E C',
 'B C D',
 'B D E',
 'C A B',
 'D A B',
 'A B C',
 'A E B',
 'E C B',
 'E D C',
 'E D A',
 'B A D',
 'C D A',
 'E D C',
 'C A D',
 'D C E',
 'D B E',
 'B E A',
 'D E A',
 'D A C',
 'E B D',
 'E D A',
 'D E B',
 'E D A',
 'E D A',
 'A D B',
 'B E C',
 'B D C',
 'E A C',
 'A B D',
 'B A D',
 'B C D',
 'A B C',
 'B A E',
 'B A C',
 'A C E',
 'A C B',
 'A C D',
 'B D A',
 'A B C',
 'C D E',
 'B C D',
 'A D B',
 'B C D',
 'A B E',
 'E A C',
 'A C B',
 'E B C',
 'C D E',
 'B C D',
 'A B C',
 'C E D',
 'B D C',
 'D A B',
 'E A C',
 'E A C',
 'B C D',
 'B C D',
 'E A C',
 'A D E',
 'A B D',
 'A C E',
 'A E B',
 'A D E',


In [12]:
predictions = train['prediction']

In [13]:
predictions.str.split(" ")

0      [A, C, E]
1      [E, B, C]
2      [A, D, E]
3      [E, A, B]
4      [D, B, A]
         ...    
195    [D, B, A]
196    [A, D, B]
197    [B, A, C]
198    [E, A, D]
199    [C, D, B]
Name: prediction, Length: 200, dtype: object

In [14]:
def average_percision(lst, ans):
    res = []
    for i in range(len(lst)):
        if lst[i] == ans:
            res.append(1/(i+1))
            # Once a correct label has been scored for an individual question in the test set, 
            # that label is no longer considered relevant for that question, 
            # and additional predictions of that label are skipped in the calculation.
            return np.sum(res)
        else:
            res.append(0)
    return 0

In [15]:
maps = []

for pred, ans in train[['prediction', 'answer']].to_numpy():
    pred = pred.split(" ")
    maps.append(average_percision(pred, ans))


In [16]:
train[['prediction', 'answer']]

,prediction,answer
0,A C E,D
1,E B C,A
2,A D E,A
3,E A B,C
4,D B A,D
...,...,...
195,D B A,C
196,A D B,B
197,B A C,B
198,E A D,D


In [17]:
np.mean(maps)

0.21333333333333332

In [18]:
maps

[0,
 0,
 1.0,
 0,
 1.0,
 0.3333333333333333,
 0.5,
 0.3333333333333333,
 1.0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.0,
 0.5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.3333333333333333,
 0,
 0,
 0,
 1.0,
 0,
 0,
 0,
 0,
 0.5,
 0,
 0,
 0.3333333333333333,
 0,
 0,
 0.5,
 0,
 0,
 0,
 0,
 0,
 0.5,
 0,
 0,
 0.3333333333333333,
 0,
 0.3333333333333333,
 0,
 0,
 0,
 0,
 1.0,
 0.3333333333333333,
 0,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.5,
 0,
 0,
 0,
 0.3333333333333333,
 0,
 0.3333333333333333,
 1.0,
 0,
 0,
 0.5,
 0.5,
 0.5,
 1.0,
 0,
 0,
 0,
 0.5,
 0.3333333333333333,
 0,
 1.0,
 0.3333333333333333,
 0,
 0,
 0,
 1.0,
 0,
 0,
 1.0,
 0,
 1.0,
 0,
 0,
 0,
 0,
 0.5,
 0,
 0.5,
 0,
 0,
 0.5,
 0,
 0,
 0,
 0.3333333333333333,
 0,
 0.3333333333333333,
 0,
 1.0,
 0.5,
 0,
 0,
 1.0,
 0,
 0.5,
 0.3333333333333333,
 0.3333333333333333,
 0,
 0.3333333333333333,
 0,
 1.0,
 0,
 0,
 0,
 0,
 0,
 1.0,
 0.3333333333333333,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.5,
 0,
 0,
 0.5,
 0,
 0,
 0.33333333333